In [ ]:
!nvidia-smi

# environment

In [ ]:
!pip install gdown --quiet

### code

In [ ]:
%%bash
File_id="1AfRVKQtmHyoDd40PatqVOTzWiQaZyFs3"
File_name="benchmark"

gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"
unzip -q "${File_name}.zip" -d "${File_name}"
# rm -f "${File_name}.zip"

In [ ]:
%cd benchmark/

### package

In [ ]:
!pip install -r requirements.txt --quiet # --q 讓他安靜

### data

In [ ]:
%%bash
File_id="1rsLlnqNiCAUt9SqzMfqVp8TwYkRhFa0z"
gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"

In [ ]:
%%bash
folder="data/train/gm12878_ctcf"

mkdir -p $folder
unzip -q "gm12878_ctcf.zip" -d $folder

ls $folder

In [ ]:
!rm -r gm12878_ctcf.zip

### Existing output

In [ ]:
%%bash
File_id="14Aush5V6iQ7MgoRlOMIPILxbezPUQpYA"
gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"

folder="output"

mkdir -p $folder
unzip -q "output.zip" -d $folder

# rename
# sour_folder="${folder}/24-5-11-basecnn-gm12878-ctcf-hyber-test-part2-1"
# tar_folder="${folder}/2024.07.07_cnn_hyper-test-basic-structure"

# mv $sour_folder $tar_folder

### run

In [ ]:
%%bash

# Set variables (use double quotes for variable expansion)
input_folder="data/train/gm12878_ctcf"
out_folder="output/2024.08.03_cnn_hyper-test-final-1000bp"
input_format="onehot"

# Fixed variables
lr=0.001
bp_range_ls=1000

# Basic structure
extractor_pool_type=max
extractor_feature_aggregation=fc
extractor_feature_agg_rate=0.5
extractor_dropout=0.3
extractor_bn=False
classifier_bn=False


# Hyperparameter options (use arrays)
extractor_hidden_size="10 50 100 200 300"
classifier_dropout="0.0" # classifier_dropout="0.0 0.5"
classifier_hidden_layer="1 2 3 5"
classifier_hidden_size="10 50 100 200 300"

# run
total=100
current_test=0

for bp in $bp_range_ls; do
  for clf_layer in $classifier_hidden_layer; do
    for clf_hs in $classifier_hidden_size; do
      for ext_hs in $extractor_hidden_size; do
        for clf_drop in $classifier_dropout; do
          # Build config and output paths
          config="conf/base-CNN_${bp}bp.yaml"
          input="${input_folder}/${bp}bp.50ms.${input_format}/data.h5"
          out="${out_folder}/Bp${bp}_Hl${clf_layer}_Hs${clf_hs}_CHs${ext_hs}_ClfDrop${clf_drop}"

          # Print information
          echo ">>>>>>>>>>>>>>>>> Task <<<<<<<<<<<<<<<<<"
          echo "Progress: $((current_test + 1)) / $total"
          echo "config: $config"
          echo "input: $input"
          echo "out: $out"
          echo "<<<<<<<<<<<<<<<<< Task >>>>>>>>>>>>>>>>>"
          echo

          # Update counter
          ((current_test++))

          # Execute training script with arguments (use single quotes for variable preservation)
          python train.py \
            --config "$config" \
            --input "$input" \
            --output_folder "$out" \
            --device gpu \
            --eval_freq 1 \
            --pin_memory_train True \
            --train.learning_rate "$lr" \
            --model.extractor_hidden_size "$ext_hs" \
            --model.extractor_output_dim "$clf_hs" \
            --model.extractor_dropout "$extractor_dropout" \
            --model.extractor_bn "$extractor_bn" \
            --model.extractor_pool_type "$extractor_pool_type" \
            --model.extractor_feature_aggregation "$extractor_feature_aggregation" \
            --model.extractor_feature_agg_rate "$extractor_feature_agg_rate" \
            --model.classifier_input_dim "$clf_hs" \
            --model.classifier_hidden_size "$clf_hs" \
            --model.classifier_hidden_layer "$clf_layer" \
            --model.classifier_dropout "$clf_drop" \
            --model.classifier_bn "$classifier_bn"

          # 檢查執行結果
          if [ $? -ne 0 ]; then
            echo "Error: Training failed -> ${out_folder}"
            exit 1
          fi
        done
      done
    done
  done
done

zip -r "${out_folder}.zip" "${out_folder}"

echo "All tasks finished."

In [ ]:
# 一定要在這路徑底下才行
import os
os.chdir('/kaggle/working')

from IPython.display import FileLink
FileLink('benchmark/output/2024.08.03_cnn_hyper-test-final-1000bp.zip')